In [ ]:
# 쿠팡 크롤링시 기본 템플릿
import os
import urllib3
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
import requests
from bs4 import BeautifulSoup

os.environ["WDM_SSL_VERIFY"] = "0"

#InsecureRequestWarning 비활성화
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
# 크롬 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.110 Safari/537.36")


# webdriver-manager로 ChromeDriver 자동 설치 및 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options = chrome_options)

# Selenium 감지 속성 제거
driver.execute_cdp_cmd(
    "Page.addScriptToEvaluateOnNewDocument",
    {
        "source": """
        Object.defineProperty(navigator, 'webdriver', {
          get: () => undefined
        })
        """
    }
)

In [ ]:
f'https://www.coupang.com/np/search?rocketAll=false&q={keyword}&filterType={filterType}&priceRange={minPrice}&minPrice={minPrice}&maxPrice={maxPrice}&page={p}&filterSetByUser=true&channel=user&&rating={star}&sorter={sorter}&listSize=36'


In [3]:
keyword = input('키워드')
page = int(input('총 페이지?'))+1
filterType = input('rocket vs free')
minPrice = input('최소 가격')
maxPrice = input('최대 가격')
star = input('별점')
sorter = input('정렬')
listSize=72

url = f'https://www.coupang.com/np/search?rocketAll=false&q={keyword}&filterType={filterType}&isPriceRange=True&minprice={minPrice}&maxPrice={maxPrice}&filterSetByUser=True&channel=user&rating={star}&sorter={sorter}&listSize=72'

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options = chrome_options)

In [ ]:
result = []

for p in range(1,page) :
    driver.get(f'https://www.coupang.com/np/search?rocketAll=false&q={keyword}&filterType={filterType}&isPriceRange=True&minprice={minPrice}&maxPrice={maxPrice}&page={p}&filterSetByUser=True&channel=user&rating={star}&sorter={sorter}&listSize=36')
    driver.implicitly_wait(5)
    items = driver.find_elements(By.CSS_SELECTOR, ".search-product:not(.search-product__ad-badge):not(.best-seller-carousel-item)")
    best = driver.find_elements(By.CSS_SELECTOR, ".search-product.best-seller-carousel-item")[:5]

# best 상품 상위 5개 result 에 추가
    if driver.find_elements(By.CSS_SELECTOR, ".search-product.best-seller-carousel-item") == [] :
        pass
    else:
        for b in best :
            try:
                name = '★ ' + b.find_element(By.CSS_SELECTOR, 'div.name').text
            except:
                name = ''
            try:
                link = b.find_element(By.CSS_SELECTOR, '.search-product > a').get_attribute('href')
            except:
                link = ''
            try:
                price = int(b.find_element(By.CSS_SELECTOR, 'strong.price-value').text.replace(',',''))
            except:
                price = ''
            try:
                star = float(b.find_element(By.CSS_SELECTOR, 'span.star > em.rating').text)
            except:
                star = '-'
            try:
                review = int(b.find_element(By.CSS_SELECTOR, 'span.rating-total-count').text.strip('(').strip(')'))
            except:
                review = '-'
        result.append([name, link, price, star, review])

# 일반 상품 result 에 추가
    for i in items :
        name = i.find_element(By.CSS_SELECTOR, 'div.name').text
        link = i.find_element(By.CSS_SELECTOR, '.search-product > a').get_attribute('href')
        price = int(i.find_element(By.CSS_SELECTOR, 'strong.price-value').text.replace(',',''))
        try:
            star = float(i.find_element(By.CSS_SELECTOR, 'span.star > em.rating').text)
        except:
            star = '-'
        try:
            review = int(i.find_element(By.CSS_SELECTOR, 'span.rating-total-count').text.strip('(').strip(')'))
        except:
            review = '-'
        result.append([name, link, price, star, review])

df = pd.DataFrame(result, columns=['상품명','링크','가격','별점','리뷰 수'])
df.to_excel(f'{keyword}.xlsx')